In [ ]:
!pip install gymnasium -q
!pip install tensorboard -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 879.1/879.1 KB 17.7 MB/s eta 0:00:00


In [ ]:
!pip install wandb -q
!wandb login

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 KB 15.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 15.7 MB/s eta 0:00:00
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
# b01af515b5a35d1f0b9fbaecfff35d74e4fd8b1e

In [ ]:
!git clone https://github.com/helpingstar/gym-snakegame.git
%cd gym-snakegame
!pip install -r requirements.txt
!pip install -e .

Cloning into 'gym-snakegame'...
remote: Enumerating objects: 62, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 62 (delta 22), reused 53 (delta 15), pack-reused 0
Unpacking objects: 100% (62/62), done.
/content/gym-snakegame
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 KB 35.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 36.5 MB/s eta 0:00:00
  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110742 sha256=a4c49e10e0b16e164de4c9bbed8d378b3b6019a482ec3cbb7e5bd685f10862b0
  Stored in directory: /root/.cache/pip/wheels/e4/a4/db/0368d3a04033da662e13926594b3a8cf1aa4ffeefe570cfac1
Successfully built moviepy
  Attempting uninstall: moviepy
    F

In [ ]:
import gymnasium as gym
import torch.nn as nn
import wandb
import time
import torch
import torch.optim as optim
import random
from tqdm import tqdm
import numpy as np
import torch.nn.functional as F
import gym_snakegame
from torch.utils.tensorboard import SummaryWriter

In [ ]:
import numpy as np
from typing import Dict
from gymnasium import spaces

def obs_space_to_shape(obs_space: spaces.Space):
    if isinstance(obs_space, spaces.Box):
        return obs_space.shape
    if isinstance(obs_space, spaces.Discrete):
        return (1,)

    else:
        raise NotImplementedError(f"{obs_space} observation space is not supported")

class ReplayBuffer:
    """A simple numpy replay buffer."""

    def __init__(self, obs_space:spaces.Space , size: int, batch_size: int = 32):
        self.max_size, self.batch_size = size, batch_size
        self.obs_shape = obs_space_to_shape(obs_space)
        self.obs_buf = np.zeros((size,) + self.obs_shape, dtype=np.float32)
        self.next_obs_buf = np.zeros((size,) + self.obs_shape, dtype=np.float32)
        self.acts_buf = np.zeros([size], dtype=np.float32)
        self.rews_buf = np.zeros([size], dtype=np.float32)
        self.done_buf = np.zeros(size, dtype=np.float32)
        self.ptr, self.size, = 0, 0

    def store(
        self,
        obs: np.ndarray,
        act: np.ndarray,
        rew: float,
        next_obs: np.ndarray,
        done: bool,
    ):
        self.obs_buf[self.ptr] = obs
        self.next_obs_buf[self.ptr] = next_obs
        self.acts_buf[self.ptr] = act
        self.rews_buf[self.ptr] = rew
        self.done_buf[self.ptr] = done
        self.ptr = (self.ptr + 1) % self.max_size
        self.size = min(self.size + 1, self.max_size)

    def sample_batch(self) -> Dict[str, np.ndarray]:
        idxs = np.random.choice(self.size, size=self.batch_size, replace=False)
        return dict(obs=self.obs_buf[idxs],
                    next_obs=self.next_obs_buf[idxs],
                    acts=self.acts_buf[idxs],
                    rews=self.rews_buf[idxs],
                    done=self.done_buf[idxs])

    def __len__(self) -> int:
        return self.size


class AddChannelDimension(gym.ObservationWrapper):
    """Image shape to num_channels x height x width"""

    def __init__(self, env):
        super(AddChannelDimension, self).__init__(env)
        shape = self.observation_space.shape
        self.observation_space = gym.spaces.Box(low=0, high=5, shape=(1, shape[0], shape[1]))

    def observation(self, observation):
        shape = observation.shape
        return observation.reshape(1, shape[0], shape[1])

class DivideObservation(gym.ObservationWrapper):
    def __init__(self, env, divide=1.0):
        super().__init__(env)
        self.divide = divide

    def observation(self, obs):
        return obs / self.divide

class RewardModifier(gym.RewardWrapper):
    def __init__(self, env):
        gym.RewardWrapper.__init__(self, env)

    def reward(self, reward):
        if reward == 0:
            return -0.5
        return reward * 20

In [ ]:
args = {
    'env_id': 'gym_snakegame/SnakeGame-v0',
    'env_id_short' : 'SnakeGame-v0',
    'seed': 42,
    'cuda': True,
    'learning_rate' : 0.003,
    'buffer_size' : 60000,
    'total_timesteps' : 8000000,
    'start_e' : 1, 
    'end_e' : 0.05, 
    'exploration_fraction' : 0.5,
    'wandb_project_name' : "dqn-Snakegame",
    'wandb_entity' : None,
    'learning_starts' : 60000,
    'train_frequency' : 1,
    'batch_size' : 256,
    'target_network_frequency' : 500,
    'gamma' : 0.95,
    'capture_video' : False
    }

device = torch.device("cuda" if torch.cuda.is_available() and args["cuda"] else "cpu")
print(device)
run_name=f"{args['env_id_short']}_{args['seed']}_{int(time.time())}"

cpu


In [ ]:
# input.shape : (5, 5)
class DQN(nn.Module):
    def __init__(self, env):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(in_features=env.observation_space.shape[0], out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=env.action_space.n)
        )

    def forward(self, x):
        return self.network(x)

In [ ]:
def linear_schedule(start_e: float, end_e:float, duration: int, t: int):
    slope = (end_e - start_e) / duration
    return max(slope * t + start_e, end_e)

In [ ]:
env = gym.make(args['env_id'], size=5, n_target=1, render_mode='rgb_array')
env = gym.wrappers.RecordVideo(env, f"/content/drive/MyDrive/RL/snakegame/video/{run_name}", 
                               episode_trigger=lambda x: x % 200 == 0,
                               name_prefix=run_name,
                               disable_logger=True)
env = AddChannelDimension(env)
env = RewardModifier(env)
env = DivideObservation(env, 5.0)
env = gym.wrappers.FlattenObservation(env)

/usr/local/lib/python3.8/dist-packages/gymnasium/utils/passive_env_checker.py:35: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (5, 5)
  logger.warn(


In [ ]:
# wandb.tensorboard.patch(root_logdir='runs')
wandb.init(
    # set the wandb project where this run will be logged
    name=run_name,
    project=args['wandb_project_name'],
    entity=args['wandb_entity'],
    # sync_tensorboard=True,
    config=args,
    monitor_gym=True,
    save_code=True
)

writer = SummaryWriter(f'/content/drive/MyDrive/RL/snakegame/tensorboard/runs/{run_name}')
writer.add_text(
    "hyperparameters",
    "|param|value|\n|-|-|\n%s" % ("\n".join([f"|{key}|{value}|" for key, value in args.items()])),
)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iamhelpingstar. Use `wandb login --relogin` to force relogin


In [ ]:
q_network = DQN(env).to(device)
optimizer = optim.Adam(q_network.parameters(), lr=args['learning_rate'])
target_network = DQN(env).to(device)
target_network.load_state_dict(q_network.state_dict())

rb = ReplayBuffer(
    env.observation_space,
    args['buffer_size'],
    args['batch_size']
)

start_time = time.time()

obs, _ = env.reset()
score = 0
episode_cnt = 0
for global_step in tqdm(range(args['total_timesteps'])):
    epsilon = linear_schedule(args['start_e'], 
                              args['end_e'], 
                              args['exploration_fraction'] * args['total_timesteps'], 
                              global_step)
    if random.random() < epsilon:
        action = env.action_space.sample()
    else:
        q_values = q_network(torch.Tensor(obs).to(device))
        action = torch.argmax(q_values).item()
    
    next_obs, reward, terminate, truncate, info = env.step(action)
    rb.store(obs, action, reward, next_obs, terminate)
    
    obs = next_obs
    score += reward 
    
    if terminate:
        obs, _ = env.reset()
        writer.add_scalar("charts/episodic_return", score, global_step)
        wandb.log({"charts/episodic_return": score}, step=global_step)
        score = 0
        episode_cnt += 1
        
    writer.add_scalar("charts/epsilon", epsilon, global_step)
    wandb.log({"charts/epsilon": epsilon}, step=global_step)
    
    if global_step > args['learning_starts']:
        if global_step % args['train_frequency'] == 0:
            
            samples = rb.sample_batch()
            states = torch.FloatTensor(samples['obs']).to(device)
            next_states = torch.FloatTensor(samples['next_obs']).to(device)
            actions = torch.LongTensor(samples['acts']).reshape(-1, 1).to(device)
            rewards = torch.FloatTensor(samples['rews']).reshape(-1, 1).to(device)
            dones = torch.FloatTensor(samples["done"].reshape(-1, 1)).to(device)
            
            with torch.no_grad():
                target_max, _ = target_network(next_states).max(dim=1)
                td_target = rewards.flatten() + args['gamma'] * target_max * (1 - dones.flatten())
            old_val = q_network(states).gather(1, actions).squeeze()
            loss = F.mse_loss(td_target, old_val)
            
            if global_step % 100 == 0:
                writer.add_scalar("losses/td_loss", loss, global_step)
                writer.add_scalar("losses/q_values", old_val.mean().item(), global_step)
                writer.add_scalar("charts/SPS", int(global_step / (time.time() - start_time)), global_step)
                
                wandb.log({"losses/td_loss": loss, "losses/q_values": old_val.mean().item()}, step=global_step)
                wandb.log({"charts/SPS" : int(global_step / (time.time() - start_time))}, step=global_step)
            # optimize the model 
            optimizer.zero_grad()
            loss.backward()  
            optimizer.step()
        
        # update the target network
        if global_step % args['target_network_frequency'] == 0:
            target_network.load_state_dict(q_network.state_dict())
env.close()
writer.close()
wandb.finish()

100%|██████████| 8000000/8000000 [16:51:30<00:00, 131.82it/s]


charts/SPS,█▅▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
charts/episodic_return,▁▁▁▂▁▂▁▂▂▁▁▂▂▁▂▂▂▁▁▁▂▂▂▁▁▂▁▂▄▁▂▂▂▄█▆▃▆▅▄
charts/epsilon,██▇▇▇▆▆▆▅▅▄▄▄▃▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
losses/q_values,▅█▇▄▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
losses/td_loss,▃█▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
charts/SPS,131
charts/episodic_return,-1.5
charts/epsilon,0.05
losses/q_values,63.46108
losses/td_loss,33.2687


In [ ]:
torch.save(q_network, f'/content/drive/MyDrive/RL/snakegame/weight/q_network_{run_name}.pt')
torch.save(target_network, f'/content/drive/MyDrive/RL/snakegame/weight/target_network_{run_name}.pt')